In [46]:
!nvidia-smi

Mon Jan 24 17:53:07 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#Setup

In [1]:
!git clone https://github.com/hungcao0402/PaddleOCR-Vietnamese

Cloning into 'PaddleOCR-Vietnamese'...
remote: Enumerating objects: 351, done.
remote: Counting objects: 100% (351/351), done.
remote: Compressing objects: 100% (292/292), done.
remote: Total 351 (delta 58), reused 337 (delta 50), pack-reused 0
Receiving objects: 100% (351/351), 12.67 MiB | 7.92 MiB/s, done.
Resolving deltas: 100% (58/58), done.


In [2]:
%cd /content/PaddleOCR-Vietnamese

/content/PaddleOCR-Vietnamese


In [ ]:
!pip install -r requirements.txt

In [ ]:
!pip install paddlepaddle-gpu

#Download data

In [4]:
!gdown --id 1UUQhNvzgpZy7zXBFQp0Qox-BBjunZ0ml
!unzip vietnamese_original.zip -d train

Downloading...
From: https://drive.google.com/uc?id=1UUQhNvzgpZy7zXBFQp0Qox-BBjunZ0ml
To: /content/PaddleOCR-Vietnamese/vietnamese_original.zip
100% 1.05G/1.05G [00:10<00:00, 97.3MB/s]


#Train

##Train model detection


Tiền xử lý dữ liệu train model detection

In [8]:
!python3 ConvertFromBtcToFormatPaddle.py

Download pretrain model SAST

In [9]:
!wget https://paddleocr.bj.bcebos.com/dygraph_v2.0/en/det_r50_vd_sast_icdar15_v2.0_train.tar -P ./pretrain_models
!tar -xf ./pretrain_models/det_r50_vd_sast_icdar15_v2.0_train.tar -C /content/PaddleOCR-Vietnamese/pretrain_models

--2022-01-24 16:41:50--  https://paddleocr.bj.bcebos.com/dygraph_v2.0/en/det_r50_vd_sast_icdar15_v2.0_train.tar
Resolving paddleocr.bj.bcebos.com (paddleocr.bj.bcebos.com)... 103.235.46.61, 2409:8c04:1001:1002:0:ff:b001:368a
Connecting to paddleocr.bj.bcebos.com (paddleocr.bj.bcebos.com)|103.235.46.61|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1041520640 (993M) [application/x-tar]
Saving to: ‘./pretrain_models/det_r50_vd_sast_icdar15_v2.0_train.tar’

det_r50_vd_sast_icd 100%[===================>] 993.27M  10.5MB/s    in 2m 2s   

2022-01-24 16:43:56 (8.16 MB/s) - ‘./pretrain_models/det_r50_vd_sast_icdar15_v2.0_train.tar’ saved [1041520640/1041520640]



Training

In [ ]:
!python3 tools/train.py -c ./configs/det/SAST.yml

##Train model recognition

Tiền xử lý dữ liệu train model recognition

In [14]:
!python3 crop_image.py

Training

In [ ]:
!python3 tools/train.py -c ./configs/rec/SRN.yml 

#Evaluation

In [ ]:
#evaluation model detection
!python3 tools/eval.py -c SAST.yml \
                       -o Global.checkpoints_model=path_checkpoints 

In [ ]:
#evaluation model recognition
!python3 tools/eval.py -c ./configs/rec/SRN.yml  \
                       -o Global.checkpoints=.path_checkpoints \
                          Global.character_type=ch  \
                          Global.character_dict_path=./ppocr/utils/dict/vi_vietnam.txt 

#Convert to inference model

In [ ]:
!python3 tools/export_model.py -c ./configs/det/SAST.yml  \
                              -o Global.pretrained_model= #path_pretrained \
                                 Global.save_inference_dir=./inference/SAST
                                 
!python3 tools/export_model.py -c ./configs/rec/SRN.yml .
                              -o Global.pretrained_model= #path_pretrained \
                                 Global.save_inference_dir=./inference/SRN

Sau khi convert xong thì predict với model trên

In [17]:
#Download model inference
!gdown --id 1slORHgS8awv7BeuSDahJvV1POPQtP_I_
!gdown --id 1-M5EMuLyFlOfp2bNMX6tibNmY3WReIxo
!unzip SAST.zip -d inference
!unzip SRN.zip -d inference

Downloading...
From: https://drive.google.com/uc?id=1slORHgS8awv7BeuSDahJvV1POPQtP_I_
To: /content/PaddleOCR-Vietnamese/SAST.zip
100% 180M/180M [00:01<00:00, 176MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-M5EMuLyFlOfp2bNMX6tibNmY3WReIxo
To: /content/PaddleOCR-Vietnamese/SRN.zip
100% 204M/204M [00:01<00:00, 134MB/s]
Archive:  SAST.zip
   creating: inference/SAST/
  inflating: inference/SAST/inference.pdmodel  
  inflating: inference/SAST/inference.pdiparams  
  inflating: inference/SAST/inference.pdiparams.info  
Archive:  SRN.zip
   creating: inference/SRN/
  inflating: inference/SRN/inference.pdiparams  
  inflating: inference/SRN/inference.pdiparams.info  
  inflating: inference/SRN/inference.pdmodel  


In [18]:
#predict detection
!python3 tools/infer/predict_det.py --det_algorithm="SAST" --use_gpu=True \
                                    --det_model_dir="./inference/SAST"  \
                                    --image_dir=./train/vietnamese/unseen_test_images/im1502.jpg

/usr/local/lib/python3.7/dist-packages/skimage/data/__init__.py:107: DeprecationWarning: 
    Importing file_hash from pooch.utils is DEPRECATED. Please import from the
    top-level namespace (`from pooch import file_hash`) instead, which is fully
    backwards compatible with pooch >= 0.1.
    
  return file_hash(path) == expected_hash
[2022/01/24 17:27:48] root INFO: im1502.jpg	[[[107, 78], [186, 90], [185, 121], [106, 114]], [[194, 91], [286, 100], [285, 128], [193, 122]], [[215, 236], [267, 236], [267, 272], [214, 273]], [[52, 233], [152, 234], [152, 268], [52, 267]], [[295, 305], [347, 303], [346, 342], [295, 343]], [[136, 306], [230, 302], [230, 341], [136, 338]], [[238, 302], [289, 300], [288, 337], [238, 338]], [[68, 304], [127, 300], [126, 338], [67, 340]], [[160, 235], [204, 231], [204, 268], [161, 268]], [[13, 302], [60, 298], [57, 340], [14, 340]]]

[2022/01/24 17:27:48] root INFO: The predict time of ./train/vietnamese/unseen_test_images/im1502.jpg: 3.6197309494018555
[20

In [19]:
#predict recognition
!python3 tools/infer/predict_rec.py --image_dir="./train/img_crop/im0001_0.jpg" \
                                    --use_gpu=True \
                                    --rec_algorithm="SRN" \
                                    --rec_model_dir="./inference/SRN/"  \
                                    --rec_image_shape="1, 64, 256"  \
                                    --rec_char_type="ch"   \
                                    --rec_char_dict_path="./ppocr/utils/dict/vi_vietnam.txt"

/usr/local/lib/python3.7/dist-packages/skimage/data/__init__.py:107: DeprecationWarning: 
    Importing file_hash from pooch.utils is DEPRECATED. Please import from the
    top-level namespace (`from pooch import file_hash`) instead, which is fully
    backwards compatible with pooch >= 0.1.
    
  return file_hash(path) == expected_hash
[2022/01/24 17:27:57] root INFO: Predicts of ./train/img_crop/im0001_0.jpg:('CHẤT', 0.99999535)


#Text Detection and Recognition Inference Concatenation

In [44]:
!python3 ./tools/infer/predict_system.py --use_gpu=True  \
    --det_algorithm="SAST"  \
    --det_model_dir="./inference/SAST"  \
    --rec_algorithm="SRN" \
    --rec_model_dir="./inference/SRN/"  \
    --rec_image_shape="1, 64, 256"  \
    --rec_char_type="ch"  \
    --drop_score=0.5  \
    --vis_font_path=font-times-new-roman.ttf \
    --image_dir="./train/vietnamese/unseen_test_images/im1501.jpg" \
    --rec_char_dict_path="./ppocr/utils/dict/vi_vietnam.txt" 

/usr/local/lib/python3.7/dist-packages/skimage/data/__init__.py:107: DeprecationWarning: 
    Importing file_hash from pooch.utils is DEPRECATED. Please import from the
    top-level namespace (`from pooch import file_hash`) instead, which is fully
    backwards compatible with pooch >= 0.1.
    
  return file_hash(path) == expected_hash
[2022/01/24 17:48:35] root DEBUG: dt_boxes num : 0, elapse : 2.23461651802063
[2022/01/24 17:48:35] root DEBUG: rec_res num  : 0, elapse : 2.384185791015625e-06
[2022/01/24 17:48:36] root DEBUG: dt_boxes num : 0, elapse : 0.9789605140686035
[2022/01/24 17:48:36] root DEBUG: rec_res num  : 0, elapse : 2.384185791015625e-06
[2022/01/24 17:48:37] root DEBUG: dt_boxes num : 0, elapse : 0.9548494815826416
[2022/01/24 17:48:37] root DEBUG: rec_res num  : 0, elapse : 2.1457672119140625e-06
[2022/01/24 17:48:38] root DEBUG: dt_boxes num : 0, elapse : 0.947263240814209
[2022/01/24 17:48:38] root DEBUG: rec_res num  : 0, elapse : 2.384185791015625e-06
[2022/01/2